In [8]:
import numpy as np
from scipy.stats import wishart

In [9]:
files_root = '/home/ruyi/cosmology/kcap_methods/bandpower_cl_methods/glass_fiducial_0/shear_cl/'

In [10]:
# bin_order = ['bin_1_1', 'bin_2_1', 'bin_3_1', 'bin_4_1', 'bin_5_1',
#              'bin_2_2', 'bin_3_2', 'bin_4_2', 'bin_5_2',
#              'bin_3_3', 'bin_4_3', 'bin_5_3',
#              'bin_4_4', 'bin_5_4',
#              'bin_5_5']

bin_order = []
for i in range(5):
    for j in range(i+1):
        bin_order.append('bin_%d_%d' %(i+1, j+1))

ell = np.genfromtxt(files_root + 'ell.txt')[1:]

To calculate the covariance, follow the instructions on the joachimi and bridlle paper page 9, just set delta l = 1, and the shape noise I can get from the kids methodology paper. Same with the galaxy number density, but have to convert from arcmins to steradians. Then the survey area is also on the kids methodology paper. Use those values, plug it all in and then have a covariance per l. To get the bandpower covariance just sum it up per l and propagate the errors.

Then for the final big combined covariance you would just have a non zero diagonal with the squares of each covariance, but lots and lots of 0 off diagonal terms as the vector from 1 bin pair doesn't affect the signal of another bin pair, so it looks something like

A 0 0 0 0 0 \
0 B 0 0 0 0 \
0 0 C 0 0 0 \
0 0 0 D 0 0 \
0 0 0 0 E 0 \
0 0 0 0 0 F

In [11]:
ellip_disp = np.array([0.27, 0.26, 0.27, 0.25, 0.27])
n_eff = np.array([0.62, 1.18, 1.85, 1.26, 1.31]) * (60 **2) * (180**2) / (np.pi**2)#in steradian converted from arcmin

In [12]:
def get_cls(l_mode_index = 0, num_tom = 5, files_root = files_root):
    ell = np.genfromtxt(files_root + 'ell.txt')[1:]
    cls = np.zeros((num_tom, num_tom))

    for i in range(num_tom):
        for j in range(num_tom):
            if j > i:
                cl = np.genfromtxt(files_root + 'bin_%s_%s.txt' %(j+1, i+1))[l_mode_index]
            else:
                cl = np.genfromtxt(files_root + 'bin_%s_%s.txt' %(i+1, j+1))[l_mode_index]
            cls[i, j] = cl
    
    return ell, cls

def resample_cls(ell, cls, l_mode_index):
    norm = 2*ell[l_mode_index] + 1
    cls_resampled = wishart.rvs(df = norm, scale = cls / norm)
    
    return cls_resampled

def bin_cls(cls, num_tom):
    cl_binned = {}

    for i in range(num_tom):
        for j in range(num_tom):
            if j <= i:
                cl_binned['bin_%s_%s' %(i+1, j+1)] = cls[i, j]
            else:
                pass
    
    return cl_binned

def add_noise(cls, bin_1, bin_2, 
              ellip_disp = np.array([0.27, 0.26, 0.27, 0.25, 0.27]), 
              gal_num = np.array([0.62, 1.18, 1.85, 1.26, 1.31]) * (60 **2) * (180**2) / (np.pi**2)):
    try:
        cl = cls['bin_{}_{}'.format(bin_1,bin_2)]
    except:
        cl = cls['bin_{}_{}'.format(bin_2,bin_1)]
    
    if bin_1 == bin_2:
        cl += (ellip_disp[int(bin_1)-1]**2) / (2*gal_num[int(bin_1)-1])

    return cl
        
def cls_calc_covariance(cls, ell, l_mode_index, A = 1006):
    pre_factor = 2 * np.pi / (A * ell[l_mode_index])
    bin_keys = list(cls.keys())
    
    cl_ref = [["null" for _ in range(len(bin_keys))] for _ in range(len(bin_keys))]
    for i, bin_1 in enumerate(bin_keys):
        for j, bin_2 in enumerate(bin_keys):
            cl_ref[i][j] = bin_1.split("_")[-2] + bin_1.split("_")[-1] + bin_2.split("_")[-2] + bin_2.split("_")[-1]
            
    cl_cov = np.zeros((len(cls), len(cls)))

    for row in range(len(cls)):
        for col in range(len(cls)):
            i = cl_ref[row][col][0]
            j = cl_ref[row][col][1]
            k = cl_ref[row][col][2]
            l = cl_ref[row][col][3]
            
            cl_cov[row][col] = add_noise(cls, i, k) * add_noise(cls, j, l) + add_noise(cls, i, l) * add_noise(cls, j, k)
    
    cl_cov *= pre_factor
    
    return cl_cov

def bandpower_cov(files_root, ell, num_tom = 5, num_bands = 8):
    total_num_ell = len(ell)
    cl_per_band = int(total_num_ell/num_bands)
    num_pairs = np.sum(np.arange(num_tom+1))
    temp_cl_cov = np.zeros((total_num_ell, num_pairs, num_pairs))
    
    for i in range(total_num_ell):
        _, theory_cls = get_cls(l_mode_index = 300, num_tom = 5, files_root = files_root)
        cl_binned = bin_cls(cls = theory_cls, num_tom = 5)
        temp_cl_cov[i] += cls_calc_covariance(cls = cl_binned, ell = ell, l_mode_index = i)
    
    combined_bandpower_cov = np.zeros((120,120))
    for i in range(num_bands):
        bandpower_cov = np.zeros((num_pairs, num_pairs))
        for j in range(cl_per_band):
            bandpower_cov += temp_cl_cov[i*num_pairs+j]
        combined_bandpower_cov[i*num_pairs:i*num_pairs + num_pairs, i*num_pairs:i*num_pairs + num_pairs] += bandpower_cov
    
    return combined_bandpower_cov

In [13]:
bandpower_cov = bandpower_cov(files_root = files_root, ell = ell)

In [ ]:
np.savetxt("/home/ruyi/cosmology/kcap_methods/bandpower_cl_methods/glass_theory_cov.txt", bandpower_cov)